## 6.일급 함수 디자인 패턴

### 6.1 사례: 전략 패턴의 리팩토링

#### 6.1.1 고전적인 전략
온라인 상점이 다음과 같은 할인 규칙을 갖고 있다고 가정한다.
<ul>
    <li> 충성도 포인트가 1,000점 이상인 고객은 전체 주문에 대해 5% 할인을 적용한다. </li>
    <li> 하나의 주문에서 20개 이상의 동일 상품을 구입하면 해당 상품에 대해 10% 할인을 적용한다. </li>
    <li> 서로 다른 상품을 10종류 이상 주문하면 전체 주문에 대해 7% 할인을 적용한다. </li>
</ul>



In [1]:
""" [예제 6-1] 플러그형 할인 전략을 가진 Order 클래스 구현 """
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity') # 이름, 충성도

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order: # 콘텍스트
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart) # 변수가 없으면 런타임에서 내부 변수를 생성
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """ 할인액을 구체적인 숫자로 반환한다"""

class FidelityPromo(Promotion): # 첫 번째 전략
    """ 충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용 """
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0 # 파이썬 3항식

class BulkItemPromo(Promotion): # 두 번째 전략
    """ 20개 이상의 동일 상품을 구입하면 10% 할인 적용 """
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion): # 세 번째 전략
    """ 10종류 이상의 상품을 구입하면 전체 7% 할인 적용 """
    def discount(self, order):
        distinct_items = {item.product for item in order.cart} # 집합을 통해 중복을 제거
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [2]:
if __name__ == "__main__":
    joe = Customer('John Doe', 0)
    ann = Customer('Ann Smith', 1100)
    cart = [
        LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)
    ]

    # 1. 충성도에 따른 할인 적용
    print(Order(joe, cart, FidelityPromo()))
    print(Order(ann, cart, FidelityPromo()))
    
    # 2. 벌크 주문의 할인 적용
    banana_cart = [LineItem('banana', 30, 0.5),
                   LineItem('apple', 10, 1.5)]
    print(Order(joe, banana_cart, BulkItemPromo()))
    
    # 3. 긴 주문의 할인 적용
    long_order = [
        LineItem(str(item_code), 1, 1.0)
        for item_code in range(10)]
    print(Order(joe, long_order, LargeOrderPromo()))
    print(Order(joe, cart, LargeOrderPromo()))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


#### 6.1.2 함수지향 전략
[예제 6-3]은 구체적인 전략을 간단히 함수로 변경했고 Promotion 추상 클래스를 제거했다.

In [3]:
""" [예제 6-3] 할인 전략을 함수로 구현한 Order 클래스 """
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity') # 이름, 충성도

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order: # 콘텍스트
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart) # iterable 값을 리스트로 변환
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart) # 변수가 없으면 런타임에서 내부 변수를 생성
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

""" 충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용 """    
def FidelityPromo(order): # 첫 번째 전략
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0 # 파이썬 3항식

""" 20개 이상의 동일 상품을 구입하면 10% 할인 적용 """
def BulkItemPromo(order): # 두 번째 전략
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

""" 10종류 이상의 상품을 구입하면 전체 7% 할인 적용 """
def LargeOrderPromo(order): # 세 번째 전략
    distinct_items = {item.product for item in order.cart} # 집합을 통해 중복을 제거
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

if __name__ == "__main__":
    joe = Customer('John Doe', 0)
    ann = Customer('Ann Smith', 1100)
    cart = [
        LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)
    ]

    # 1. 충성도에 따른 할인 적용
    print(Order(joe, cart, FidelityPromo))
    print(Order(ann, cart, FidelityPromo))
    
    # 2. 벌크 주문의 할인 적용
    banana_cart = [LineItem('banana', 30, 0.5),
                   LineItem('apple', 10, 1.5)]
    print(Order(joe, banana_cart, BulkItemPromo))
    
    # 3. 긴 주문의 할인 적용
    long_order = [
        LineItem(str(item_code), 1, 1.0)
        for item_code in range(10)
    ]
    print(Order(joe, long_order, LargeOrderPromo))
    print(Order(joe, cart, LargeOrderPromo))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


#### 6.1.3 최선의 전략 선택하기 : 단순한 접근법
[예제 6-4]의 테스트와 동일한 고객, 쇼핑 카드가 주어졌을 때 [예제 6-5]와 같이 테스트 3개를 추가해보자.

In [4]:
promos = [FidelityPromo, BulkItemPromo, LargeOrderPromo]

def best_promo(order):
    """ 최대로 할인받을 금액을 반환한다. """
    return max(promo(order) for promo in promos)

print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

<Order total: 10.00 due: 9.30>
<Order total: 30.00 due: 28.50>
<Order total: 42.00 due: 39.90>


#### 6.1.4 모듈에서 전략 찾기
새로운 할인 전략을 추가하려면 함수를 코딩하고 이 함수를 promos 리스트에 추가해야 한다. 이를 globals() 함수를 활용하여 자동화할 수 있다. global()은 현재 전역 심벌 테이블을 나타내는 딕셔너리 객체를 반환한다. 이 딕셔너리는 언제나 현재 모들에 대한 내용을 담고 있다.[예제 6-7]은 globals()에 약간 꼼수를 부려 best_promo()가 자동으로 다른 *_promo() 함수를 찾아내게 한다.

In [5]:
""" [예제 6-7] 모듈 전역 네임스페이스를 내부 조사해서 만든 promos 리스트 """
promos = [
    globals()[name]
    for name in globals()
    if name.endswith('Promo') and name != 'best_promo'
]
print(promos) # 예제 6-4의 함수 리스트와 동일하다.

[<function FidelityPromo at 0x7f6c5c5048c8>, <function BulkItemPromo at 0x7f6c5c4e2510>, <function LargeOrderPromo at 0x7f6c5c4e2598>]


promotions라는 별도 모듈에 내부 조사를 수행해서 만든 전략 함수 리스트를 사용할 수도 있다.

In [6]:
""" [예제 6-8] 새로운 promotions 모듈을 내부 조사해서 만든 promos 리스트 """
import inspect

class promotions:
    """ 충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용 """    
    def FidelityPromo(order): # 첫 번째 전략
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0 # 파이썬 3항식

    """ 20개 이상의 동일 상품을 구입하면 10% 할인 적용 """
    def BulkItemPromo(order): # 두 번째 전략
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

    """ 10종류 이상의 상품을 구입하면 전체 7% 할인 적용 """
    def LargeOrderPromo(order): # 세 번째 전략
        distinct_items = {item.product for item in order.cart} # 집합을 통해 중복을 제거
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]
print(promos)

[<function promotions.BulkItemPromo at 0x7f6c5c4e27b8>, <function promotions.FidelityPromo at 0x7f6c5c4e2840>, <function promotions.LargeOrderPromo at 0x7f6c5c4e2a60>]


### 6.2 명령
명령 패턴의 목적은 연산을 실행하는 객체(호출자<sup>Invoker</sup>)와 연산을 구현하는 객체(수신자<sup>Receiver</sup>)를 분리하는 것이다. 기본 개념은 명령 객체를 수신자와 호출자 사이에 놓고 명령은 excute(), 단 하나의 메서드로 인터페이스를 구현한다. 이런 방식을 사용하면, 호출자는 수신자의 인터페이스를 알 필요가 없고 명령의 서브 클래스를 통해 서로 다른 수신자를 추가할 수 있다. 

파이선은 호출자에 Command 객체 대신 간단히 함수를 바로 지정할 수 있다. command.execute()를 호출하는 대신, 호출자는 단지 command()을 호출하면 된다. MacroCommand는 \_\_call\_\_( ) 매서드를 가진 클래스로 구현할 수 있다. MacroCommand의 개체는 콜러블이 되며, 각 객체는 [예제 6-9]에서 보는 것 처럼 향후에 호출할 함수들의 리스트를 가진다.

In [ ]:
class MacroCommand:
    """ 명령 리스트를 실행하는 명령 """
    
    def __init__(self, commands):
        self.commands = list(commands) # 반복 가능한 객체임이 보장됨, 명령에 대한 참조를 복사함.
        
    def __call__(self):
        for command in self.commands:
            command()